<a href="https://colab.research.google.com/github/dtabuena/Patch_Ephys/blob/main/abf_handling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:




def catalogue_recs(file_loc,cell_id_order):
    'Read metadata from abf files stored in chosen folder and assigns'
    'them to a dataframe for further processing. All further abf analyses'
    'read files from this df and report values in the df.'

    file_list = get_sub_files(file_loc)
    # file_list = [file_loc+'/'+f for f in file_list]

    file_list=[f for f in file_list if '.abf' in f]

    abf_recordings_df = pd.DataFrame(data = file_list, columns=['file_name'])
    abf_recordings_df = abf_recordings_df.set_index('file_name')

    abf_recordings_df['Recording_name'] = None
    abf_recordings_df['cell_id'] = None
    for c in cell_id_order:
        abf_recordings_df[c] = None

    abf_recordings_df["protocol"] = None
    abf_recordings_df["abf_timestamp"] = None
    abf_recordings_df["channelList"] = None


    for r in np.arange(len(abf_recordings_df)):
        try:
            row_filename = abf_recordings_df.index[r]
            if '.sta' in row_filename:
                continue
            base_name = os.path.basename(row_filename)
            abf_recordings_df.loc[row_filename,'Recording_name'] = base_name
            split_words = base_name.split('_')
            re_code = ['_'+split_words[i] for i in range(len(cell_id_order))]
            re_code = ''.join(re_code)[1:]
            abf_recordings_df.loc[row_filename,'cell_id'] = re_code
            for ci in range(len(cell_id_order)):
                abf_recordings_df.loc[row_filename,cell_id_order[ci]] = split_words[ci]

            abf = pyabf.ABF(row_filename)
            abf_recordings_df.loc[row_filename,'protocol'] = abf.protocol
            abf_recordings_df.at[row_filename,'channelList'] = abf.channelList
            abf_recordings_df.at[row_filename,'abf_timestamp'] = abf.abfDateTimeString
        except:
            print(f'ERROR on :{row_filename}')
    abf_recordings_df.sort_values('file_name',inplace=True)
    protocol_set = list(set(abf_recordings_df['protocol']))
    return abf_recordings_df, protocol_set

